
# Title: 20201107_A_Y_Theatre_v1.py



# I) Import Packages

In [55]:
import os
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from typing import List, Any
from datetime import datetime
from urllib.request import Request, urlopen
pd.set_option('display.max_rows', 500)

# II) Function definition

In [46]:
def get_title_stageplay(s):
    t = s.find('meta',{'property':"og:title"})
    if t==None: 
        return None
    else: 
        t = t.get('content')
        res = t.split(sep=' at ')[0]
        res = res.replace('REVIEW: ','')
        if len(res.split(": ")) >1: 
            res = res.split(": ")[1].split(",")[0]
            return res
        else:
            return None

def get_theatre_stageplay(s):
    t = s.find('meta',{'property':"og:title"})
    if t==None: 
        return None
    else:
        if len(str(t).split(', ')) > 1:
            res = str(t).split(', ')[1].split(" -")[0]
            return res
        else:
            return None

#def get_author_review(s):
#    list_span = []
#    if s.find_all('span',style="font-family: verdana;") == []:
#        return None
#    else: 
#        for span in s.find_all('span',style="font-family: verdana;"):
#            if span.text == None:
#                list_span.append('')
#            else:
#                list_span.append(span.text)
#        review_by = str([i for i in list_span if 'Review by ' in i][0])
#        res = review_by.split('Review by ')[1]
#        return res

def get_author_review(s):
    if len(str(s).split("</noscript>")) > 1:
        split = str(s).split("</noscript>")[1].split("</a>")[0]
        res = split
        return res
    else:
        return None

def get_date_review(s):
    x = s.find('span',{'class':"cb-date"})
    date_string = x.text
    d = datetime.strptime(date_string, '%B %d, %Y')
    date_0 = d.strftime('%d/%m/%Y')
    if date_0==None: 
        return None
    else: 
        return date_0

def get_rating_review(s):
    
    if s.find('span',{'class':'score'}) == None:
        return None
    else:
        x = s.find('span',{'class':'score'})
        res = int(float(x.text)) * '★'
        return res

def get_review(s): 
    text = ""
    for i in s.findAll("p"):
        text = text + i.text
    text = text.replace('\xa0',"")
    return text


# III) Scrapping

In [35]:
list_link_reviews = []
for j in range(1,68):
    url = "https://www.ayoungertheatre.com"
    section = f"/category/reviews/westend//page/{j}/"
    req = Request(url+section, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    webpage = webpage.decode('utf-8')
    soup = BeautifulSoup(webpage, 'html.parser')
    a_content = soup.findAll("a",href=True)
    a_list= list(a_content)
    for i in range(0,len(a_list)):
        if "review-" in a_list[i]['href']:
            list_link_reviews.append(a_list[i]['href'])
list_link_reviews = [item for item in list_link_reviews if '#' not in item]


We are scrapping 1320 reviews


In [36]:
list_link_reviews = list(set(list_link_reviews))
print(f'We are scrapping {len(list_link_reviews)} reviews')


We are scrapping 395 reviews


# IV) Dataset Creation

In [43]:
# Initialization of lists:
title = []
reviewer = []
review = []
date = []
theatre = []
rating = []
newspaper = []

In [47]:
#Store in lists title, date , reviewer , theatre and review of comedy
for k in range(354,len(list_link_reviews)): 
#for k in range(0,20): 
    print(list_link_reviews[k])
    req = Request(list_link_reviews[k], headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    webpage = webpage.decode('utf-8')
    s = BeautifulSoup(webpage, 'html.parser')
    title.append(get_title_stageplay(s))
    date.append(get_date_review(s))
    reviewer.append(get_author_review(s))
    theatre.append(get_theatre_stageplay(s))
    review.append(get_review(s))
    rating.append(get_rating_review(s))
newspaper = ['A Younger Theatre'] * (k+1)
#rating = list_ratings[:k+1]

https://www.ayoungertheatre.com/review-the-importance-of-being-earnest-vaudeville-theatre/
https://www.ayoungertheatre.com/review-relative-values-harold-pinter-theatre/
https://www.ayoungertheatre.com/review-victory-condition-the-royal-theatre/
https://www.ayoungertheatre.com/review-precious-little-talent-ella-hickson-trafalgar-studios/
https://www.ayoungertheatre.com/review-the-bodyguard-musical-adelphi-theatre/
https://www.ayoungertheatre.com/review-behind-the-beautiful-forevers-national-olivier/
https://www.ayoungertheatre.com/review-burn-the-floor-strictly-come-dancing-shaftesbury-avenue/
https://www.ayoungertheatre.com/review-abigails-party-mike-leigh-wyndhams-theatre/
https://www.ayoungertheatre.com/review-loves-labours-lost-theatre-royal-haymarket/
https://www.ayoungertheatre.com/review-passion-play-duke-of-yorks-theatre/
https://www.ayoungertheatre.com/review-the-inheritance-noel-coward-theatre/
https://www.ayoungertheatre.com/review-the-illusionists-shaftesbury-theatre/
https:

In [48]:
# Check length of arrays 
display(len(rating))
display(len(date))
display(len(reviewer))
display(len(title))
display(len(theatre))
display(len(review))

395

395

395

395

395

395

In [49]:
# Dataset creation 
df_theatre = pd.DataFrame({'Newspaper':newspaper,
                      'Date':date,
                      'Reviewer':reviewer,
                      'Title':title,
                      'Theatre':theatre,
                      'Rating':rating,
                      'Review':review
                      })

In [57]:
df_theatre.head(30)

,Newspaper,Date,Reviewer,Title,Theatre,Rating,Review
0,A Younger Theatre,21/06/2016,Hattie Pierce,The Flying Lovers of Vitebsk,The Sam Wanamaker Playhouse,None,The constancy of Marc and Bella Chagall’s love...
1,A Younger Theatre,17/12/2014,Carla Turner,Cats,London Palladium,None,The Palladium is currently hostingthe 12-week ...
2,A Younger Theatre,28/08/2015,Felix Clutson,Tamburlaine the Great,"Tristan Bates Theatre"" property=""og:title""/>",None,Christopher Marlowe’s sweeping epic is given a...
3,A Younger Theatre,16/10/2017,Jessica Handscomb,Ink,Duke of York's,None,“It’s just a bit of fun” – a phrase that almos...
4,A Younger Theatre,04/09/2014,Jemma Wilson,Lord of the Dance,London Palladium,None,Since its official debut in 1996 at the Point ...
5,A Younger Theatre,05/12/2013,Rebecca Pinnington,Henry V,Noel Coward Theatre,None,In the final production of his season of five ...
6,A Younger Theatre,02/04/2012,Eleanor Turney,The King's Speech - A Younger Theatre,None,None,"For a play about not very much, The King’s Spe..."
7,A Younger Theatre,12/08/2015,Lydia Lakemoore,My Children! My Africa!,"Trafalgar Studios"" property=""og:title""/>",None,This intimate three-hander centres around Mr M...
8,A Younger Theatre,13/09/2015,Mimi Launder,Mouthful,"Trafalgar Studios"" property=""og:title""/>",None,Mouthful is a relentlessly harrowing reminder...
9,A Younger Theatre,15/12/2018,Suzannah Ball,The Messiah,The Other Palace,None,The Messiah is a hilariously unpredictable exp...


In [62]:
df_theatre_v2 = df_theatre.copy()
df_theatre_v2['Title'] = df_theatre_v2['Title'].apply(lambda x: x.replace('- A Younger Theatre','') if x != None else x)
df_theatre_v2['Theatre'] = df_theatre_v2['Theatre'].apply(lambda x: x.replace('" property="og:title"/>','') if x != None else x)

In [64]:
df_theatre_v2.head(300)

,Newspaper,Date,Reviewer,Title,Theatre,Rating,Review
0,A Younger Theatre,21/06/2016,Hattie Pierce,The Flying Lovers of Vitebsk,The Sam Wanamaker Playhouse,None,The constancy of Marc and Bella Chagall’s love...
1,A Younger Theatre,17/12/2014,Carla Turner,Cats,London Palladium,None,The Palladium is currently hostingthe 12-week ...
2,A Younger Theatre,28/08/2015,Felix Clutson,Tamburlaine the Great,Tristan Bates Theatre,None,Christopher Marlowe’s sweeping epic is given a...
3,A Younger Theatre,16/10/2017,Jessica Handscomb,Ink,Duke of York's,None,“It’s just a bit of fun” – a phrase that almos...
4,A Younger Theatre,04/09/2014,Jemma Wilson,Lord of the Dance,London Palladium,None,Since its official debut in 1996 at the Point ...
5,A Younger Theatre,05/12/2013,Rebecca Pinnington,Henry V,Noel Coward Theatre,None,In the final production of his season of five ...
6,A Younger Theatre,02/04/2012,Eleanor Turney,The King's Speech,None,None,"For a play about not very much, The King’s Spe..."
7,A Younger Theatre,12/08/2015,Lydia Lakemoore,My Children! My Africa!,Trafalgar Studios,None,This intimate three-hander centres around Mr M...
8,A Younger Theatre,13/09/2015,Mimi Launder,Mouthful,Trafalgar Studios,None,Mouthful is a relentlessly harrowing reminder...
9,A Younger Theatre,15/12/2018,Suzannah Ball,The Messiah,The Other Palace,None,The Messiah is a hilariously unpredictable exp...


In [65]:
df_theatre_v2.tail(3)

,Newspaper,Date,Reviewer,Title,Theatre,Rating,Review
392,A Younger Theatre,07/02/2016,Guy Withers,The Magic Flute,ENO,None,Collaboration in theatre only makes for strong...
393,A Younger Theatre,06/11/2020,Ryan Van Champion,Philip Pullman’s Grimm Tales,Unicorn Theatre,★★★★★,Award winning children’s theatre The Unicorn i...
394,A Younger Theatre,20/12/2015,Mimi Launder,The Wasp,Trafalgar Studios,None,"The Wasp starts innocently enough: a cafe, cha..."


In [66]:
# Export to csv and excel format
df_theatre_v2.to_csv('20201109_ayoungertheatre_v1.csv')
df_theatre_v2.to_excel('20201109_ayoungertheatre_v1.xlsx')